In [ ]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('1IylB4N87jqhvi6yF4vOE-fg12AjymoCE')

In [ ]:
project_name= 'neural-network-number-image'

In [ ]:
!pip install jovian --upgrade --quiet

In [ ]:
# Importing necessary libraries
import torch
import pandas as pd
import numpy as np
from torchvision.datasets import MNIST
import torch.nn.functional as F
import torch.nn as nn
import torchvision as t
import matplotlib.pyplot as plt
import torchvision.transforms as transform
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

%matplotlib inline

In [ ]:
# Downloading the dataset
image_data= MNIST(root= 'C:\\Users\\L A K S H A Y\\Program files\\Dataset', download= True, transform= transform.ToTensor())

In [ ]:
# Number of enteries (lenght of the dataset)
len(image_data)

In [ ]:
# Creating the training and validation dataset
train_size= 50000
val_size= len(image_data)- train_size
train, valid= random_split(image_data, [train_size, val_size])

In [ ]:
# Some Image from the dataset
image, label= image_data[0]
plt.imshow(image[0], cmap= 'gray')
print('Label: ', label)

In [ ]:
# Example 2
image, label= image_data[100]
plt.imshow(image[0], cmap='gray')
print('Label: ', label)

In [ ]:
# Create the data loaders
train_loader= DataLoader(image_data, batch_size= 125, shuffle= True)
val_loader= DataLoader(image_data, batch_size= 125*2)                     # We can increase the size of the batch from validation dataset

In [ ]:
# Making grid of the images in the dataset
from torchvision.utils import make_grid

for image, label in train_loader:
  plt.figure(figsize=(10,8))
  print('image shape: ', image.shape)
  plt.imshow(make_grid(image, nrow= 15).permute((1,2,0)))
  break

In [ ]:
input_size= 28*28
hidden_size= 32
output_size= 10 

# Model (neural network)

In [ ]:
# In this model we are extending the nn.module 

class MNISTModel(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):              # defining the hidden layer and the output layer of the neural network 
    super().__init__()
    # 1st layer (Hidden layer)
    self.layer1= nn.Linear(input_size, hidden_size)
    # 2nd layer (outputs layer
    self.layer2= nn.Linear(hidden_size, output_size)
  
  # Defing the forward method
  def forward(self, xb):
    xb= xb.view(xb.size(0), -1)       # Flatten out the image
    out= self.layer1(xb)              # Passing the data in 1st layer (Hidden layer)
    out= F.relu(out)                  # Applying activation function
    out= self.layer2(out)             # Passing the data in layer 2 of the nerual network
    return out 

  # Training Phase
  def training_step(self, batch):
    image, label= batch
    out= self(image)
    loss= F.cross_entropy(out, label)
    return loss
  
  # Validation Phase
  def validation_step(self, batch):
    image, label= batch
    out= self(image)
    loss= F.cross_entropy(out, label)
    acc= accuracy(out, label)
    return {'validation_loss': loss, 'validation_accuracy': acc}

  # Combining the validation dataset result
  def validation_epochend(self, outputs):
    batch_loss= [x['validation_loss']for x in outputs]
    epoch_loss= torch.stack(batch_loss).mean()                                                  # Combining losses
    batch_acc= [x['validation_accuracy']for x in outputs]
    epoch_acc= torch.stack(batch_acc).mean()                                                   # Combining accuracy
    return {'validation_loss': epoch_loss.item(), 'validation_accuracy': epoch_acc.item()} 
  
  # To Print performance of the model
  def epoch_end(self, epoch, result):
    print('Epoch [{}], validation loss: {:.4f}, validation accuracy: {:.4f}'.format(epoch, result['validation_loss'], result['validation_accuracy']))

# Defining the fit method

In [ ]:
def evaluate(model, val_loader):
  outputs= [model.validation_step(batch) for batch in val_loader]
  return model.validation_epochend(outputs)

In [ ]:
def accuracy(outputs, labels):
  _, preds= torch.max(outputs, dim=1)
  return torch.tensor(torch.sum(preds == labels).item() / len(labels))

def fit(epochs, lr, model, train_loader, val_loader, opt_func= torch.optim.SGD):
  result_history= []
  optimizer= opt_func(model.parameters(), lr)           # Optimizer function
  
  for epoch in range(epochs):
    
    # Training phase
    for batch in train_loader : 
      loss= model.training_step(batch)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    
    # Validation phase
    result= evaluate(model, val_loader)
    model.epoch_end(epoch, result)
    result_history.append(result)

  return result_history

# Using GPUs
Using GPU to train your model faster

In [ ]:
# Checking your model using GPU cuda 
torch.cuda.is_available()

In [ ]:
# Checking which device is available
def get_device():
  if torch.cuda.is_available():         # If GPU is available we use GPU
    return torch.device('cuda')
  else:
    return torch.device('cpu')        # Else we will use cpu 

In [ ]:
device= get_device()
device

In [ ]:
# Moving data on available device
def to_device(data, device):
  if isinstance(data, (list,tuple)):
    return [to_device(x,device) for x in data]
  return data.to(device, non_blocking= True)

In [ ]:
# Device data loader to load your data directly on the device which is available
class DeviceDataLoader():
  def __init__(self, dataloader, device):
    # Storing the dataloader object and device in the property of the DeviceDataLoader class
    self.dl = dataloader
    self.device= device
  
  # __iter__ method to move batch of the data on the given device
  def __iter__(self):
    for xb in self.dl:
      yield to_device(xb, device)

  # __len__ to print out the number of batches
  def __len__(self):
    return len(self.dl)

## Creating the instance of the MNISTModel class or creating model

In [ ]:
model= MNISTModel(input_size, hidden_size, output_size)

In [ ]:
# Creating traning loader and validation loader on gpu
train_loader= DeviceDataLoader(train_loader, device)
val_loader= DeviceDataLoader(val_loader, device) 

In [ ]:
# Move this model on GPU
to_device(model, device)

In [ ]:
# Intial accuracy and the loss of the model
result= [evaluate(model, val_loader)]
result

# Calling fit method to train the model

In [ ]:
history1= fit(10,0.1, model, train_loader, val_loader)

In [ ]:
history2= fit(5, 0.1, model, train_loader, val_loader)

In [ ]:
history3= fit(5, 0.1, model,train_loader, val_loader)

In [ ]:
history3= fit(5, 0.1, model,train_loader, val_loader)

In [ ]:
history4= fit(5, 0.1, model,train_loader, val_loader)

In [ ]:
jovian.commit(project= project_name, environment= None)

# Apply model on test data

In [ ]:
# Download the test dataset
test_image= MNIST(root= 'C:\\Users\\L A K S H A Y\\Program files\\Dataset', train= False, transform= transform.ToTensor())

In [ ]:
# Example 1
image,label= test_image[0]
plt.imshow(image[0], cmap= 'gray')
print('Label: ',label)

In [ ]:
# Function to predict the label of the image
def predict_imagelabel(image, model):
  xb= to_device(image.unsqueeze(0), device)
  yb= model(xb)
  _, pred= torch.max(yb, dim=1)
  return pred[0].item()

In [ ]:
# Generating the prediction from the image in test data
image, label= test_image[0]
plt.imshow(image[0], cmap= 'gray')
print('Label : ', label, 'Predicted value : ', predict_imagelabel(image, model))

In [ ]:
# Example 2
image, label= test_image[182]
plt.imshow(image[0], cmap= 'gray')
print('Label : ', label, 'Predicted value : ', predict_imagelabel(image, model))

In [ ]:
# Example 3
image, label= test_image[1820]
plt.imshow(image[0], cmap= 'gray')
print('Label : ', label, 'Predicted value : ', predict_imagelabel(image, model))

In [ ]:
# Example 4
image, label= test_image[8220]
plt.imshow(image[0], cmap= 'gray')
print('Label : ', label, 'Predicted value : ', predict_imagelabel(image, model))

# Analysis the model using test data

In [ ]:
# Taking the accuracy and loss of the model on test dataset
test_loader= DeviceDataLoader(DataLoader(test_image, batch_size= 256), device)
result= evaluate(model, test_loader)
result

In [ ]:
# Saving validation loss and accuracy of the model 
jovian.log_metrics(test_loss= result['validation_loss'], test_accuracy= result['validation_accuracy'])

In [ ]:
# Saving weight and bias of the model
torch.save(model.state_dict, 'mnist-neural.pth')

In [ ]:
# Saving notebook on jovian platform
jovian.commit(project= project_name, environment=None, outputs= ['mnist-neural.pth'])